In [1]:
import h5py
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [2]:
#find path of the file
def position(path):
    files = os.listdir(path)  # get file name
    position = []
    for file in files:  # Traversing folders
        position.append(path + '\\' + file)  
    return position

In [3]:
#read file
def read(file):
    f=[]
    for i in range(len(file)):
        f.append(h5py.File(file[i],'r'))
    return f

In [4]:
#save data
def save_variable(v,filename):
    f=open(filename,'wb')
    pickle.dump(v,f)
    f.close()
    return filename

In [5]:
#read data
def load_variable(filename):
   f=open(filename,'rb')
   r=pickle.load(f)
   f.close()
   return r
def readdata(filename1):
    filename1 = load_variable(f'{filename1}.txt') 
    filename1 = np.array(filename1)
    print(filename1.shape)
    return filename1

In [6]:
def plotdata(single_grid):
    title = ["NE", "SE", "SW", "NW"] 
    colors = ["blue", "orange", "green", "red"]
    
    plt.figure(figsize=(12, 8))  
    
    for i in range(4):
        plt.plot(single_grid[:, i], label=title[i], color=colors[i], linewidth=1.5)  
    
    
    plt.ylabel("Traffic Volume", fontsize=14)  
    plt.xlabel("Timestep", fontsize=14) 
    plt.title("Traffic Flow Over Time (All Directions)", fontsize=16) 
    
    plt.legend(fontsize=12) 
    plt.grid(True, linestyle="--", alpha=0.5) 
    plt.tick_params(axis='both', which='major', labelsize=12)  
    plt.show()  # 显示图表

In [7]:
# def get_dtw_samples(f):
#     samples = []
#     for file in f:
#         if '2019-03-' in file.filename:
#             date_part = file.filename.split('_')[0]  
#             day = int(date_part.split('-')[2])  
            
#             if 16 <= day <= 22:
#                 samples.append(file)
#     return samples

In [8]:
def get_dtw_samples(f):
    samples = []
    for file in f:
        if '2020-04-' in file.filename:
            date_part = file.filename.split('_')[0]  
            day = int(date_part.split('-')[2])  
            
            if 16 <= day <= 30:
                samples.append(file)
    return samples

In [9]:
path = r"E:\Techniche University of Munich\master thesis\ANTWERP\ANTWERP\training"  
file = position(path) 
f=read(file)

In [10]:
march_second_week_samples = get_dtw_samples(f)

In [11]:
# ========== 1. 提取数据 & 计算总流量 ==========
Qd_Grid = []

for i in range(len(march_second_week_samples)):  # 遍历 5 天
    for key in march_second_week_samples[i].keys():
        
        data = march_second_week_samples[i][key][:]  # 提取数据，形状 (288, 495, 436, 8)
        data = np.array(data)

        # 选取四个方向的流量数据 (288, 495, 436, 4)
        traffic_data = data[..., [0, 2, 4, 6]]

        # **沿最后一维求和**，得到 (288, 495, 436) 形状的数据
        dtw_march_second_week_samples = traffic_data.sum(axis=3)

        Qd_Grid.append(dtw_march_second_week_samples)  # **存入列表**

# **合并所有天数的数据**，变成 `(5*288, 495, 436)`
Qd_Grid = np.concatenate(Qd_Grid, axis=0)  # (5*288, 495, 436)

# ========== 2. 转换为 30 分钟时间步 ==========
# 计算新的时间步数：每 6 个 5 分钟数据合并为 1 个 30 分钟数据
new_time_steps = Qd_Grid.shape[0] // 6  # 计算新的时间步总数

# **将数据 reshape 为 (new_time_steps, 6, 495, 436)，然后沿时间维度求和**
Qd_Grid_30min = Qd_Grid[:new_time_steps * 6].reshape(new_time_steps, 6, 495, 436).sum(axis=1)

# **最终形状： (15*48, 495, 436)**
print("最终数据形状:", Qd_Grid_30min.shape)  # 预期输出 (15*48, 495, 436)


最终数据形状: (720, 495, 436)


In [12]:
Qd_Grid_array = np.array(Qd_Grid_30min)
print(Qd_Grid_array.shape)

(720, 495, 436)


In [13]:
#Save file
save_path = r"E:\Techniche University of Munich\master thesis\ANTWERP\preprocessed_data\sliding_window\15days\30min"
filename = save_variable(Qd_Grid_30min,'Antwerp_2020april_last_half_month_samples_30min_timesteps.txt')

In [14]:
##read data
#Antwerp_traffic_january2019=readdata("Antwerp_dtw_march_second_week_samples")
#Antwerp_traffic_january2019